In [6]:
#https://github.com/gaussic/text-classification-cnn-rnn
from py2neo import Graph
from pandas import DataFrame

graph = Graph(password="neo4j")
k = DataFrame(graph.data("MATCH (a:Person) RETURN a.name, a.born"))

In [7]:
#為了方便後面做Index用，建立a-z,A-Z的26*2個英文字母，加上5個特殊符號，共57個符號
import string
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)

#建立函式方便尋找all_letters的Index像是"a"=10000000000....
def letterToIndex(letter):
    return all_letters.find(letter)

# 將一個字母轉成Tensor顯示
def letterToTensor(letter):
    tensor = np.zeros((1, n_letters)) #建立一個1*57的Tensor
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# 建立one-hot letter vectors
def lineToTensor(line):
    tensor = np.zeros((len(line), 1, n_letters))
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor

#print(letterToTensor('b')) #將內容轉為字母 #ex. a,b,c..
#print(lineToTensor('Jones')) #5為字母個數、1為維度、57為總字母

In [9]:
a = lineToTensor(k['a.name'][0][0])[0][0]
for number in range(1,len(k['a.name'])):
    ltt = lineToTensor(k['a.name'][number][0])
    b = ltt[0][0]
    a = np.vstack((a, b))

In [42]:
import tensorflow as tf
import numpy as np

# Mini-batch: instance 0,instance 1,instance 2,instance 3
#X0_batch = np.array([[0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0], [0, 0, 1, 0]]) # t = 0
#X1_batch = np.array([[0, 1, 0, 0], [0, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0]]) # t = 1
X0_batch = a[0:133]
X1_batch = a[133:266]

n_inputs = 57
n_neurons = 5
X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])
Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons],dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons,n_neurons],dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))
Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.argmax(tf.nn.softmax(tf.tanh(tf.matmul(Y0, Wy) + tf.matmul(X1, Wx) + b)))
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})
    #print(Y1_val)
    #r_eval = 
    print(Y1_val)


[  5   9  44  33 106]


In [45]:
a[Y1_val]

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
 

In [47]:
all_letters

"abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ .,;'"

In [78]:
index_Y1_val = np.where(a[Y1_val] == 1)[1]
result = k['a.name'][[index_Y1_val[0]]]

In [144]:
str(result)[6:8]

'Jo'

In [145]:
k.loc[k['a.name'].str.contains(str(result)[6:8])]

,a.born,a.name
29,1996.0,Jonathan Lipnicki
34,1966.0,John Cusack
57,1950.0,John Patrick Stanley
65,1968.0,Orlando Jones
87,1940.0,John Hurt
90,1960.0,John Goodman
106,1965.0,John C. Reilly
131,1952.0,Joel Silver
139,1952.0,Joel Silver
170,1996.0,Jonathan Lipnicki
